In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer.primitives import Sampler
import numpy as np


def setancilla1(circuit, qreg, target):
    ancilla_index = len(target)
    circuit.x(qreg[ancilla_index])
    circuit.h(qreg[ancilla_index])
    circuit.barrier()
    return circuit

def setancilla2(circuit, qreg, target):
    ancilla_index = len(target)
    circuit.h(qreg[ancilla_index])
    circuit.x(qreg[ancilla_index])
    circuit.barrier()
    return circuit


def oracle(circuit, qreg, target):
    num_qubits = len(target) + 1  
    ancilla_index = len(target)  

    rev_target = target[::-1]

    for i, bit in enumerate(rev_target):
        if bit == '0':
            circuit.x(qreg[i])
    
    circuit.mcx(list(range(len(target))), qreg[ancilla_index])
    
    for i, bit in enumerate(rev_target):
        if bit == '0':
            circuit.x(qreg[i])

    circuit.barrier()
    return circuit

def inversion(circuit, qreg, target):
    ancilla_index = len(target)
    for i in range(ancilla_index):
        circuit.h(qreg[i])
        circuit.x(qreg[i])
    
    circuit.mcx(list(range(len(target))), qreg[ancilla_index])

    circuit.x(qreg[ancilla_index])
    
    for i in range(ancilla_index):
        circuit.x(qreg[i])
        circuit.h(qreg[i])

    circuit.barrier()
    return circuit


target = "10001011"
ancilla_index = len(target)
qubits = len(target)

qreg = QuantumRegister(qubits+1)
creg = ClassicalRegister(qubits)
qc = QuantumCircuit(qreg, creg)

iterations = int(np.sqrt(2**qubits) * (np.pi/4))

for i in range(ancilla_index):
    qc.h(qreg[i])

qc = setancilla1(qc, qreg, target)

for _ in range(iterations):
    qc = oracle(qc, qreg, target)
    qc = inversion(qc, qreg, target)
   
qc = setancilla2(qc, qreg, target)

for i in range(len(target)):
    qc.measure(qreg[i], creg[i])

sampler = Sampler()
result = sampler.run(qc, shots=1000).result()
quasi_dists = result.quasi_dists[0]

print("\nProbabilitys:")
for state, probability in sorted(quasi_dists.items(), key=lambda x: -x[1]):
    binary_state = format(state, f'0{qubits}b')
    print(f"state: {binary_state}, probability: {probability:.4f}")

qc.draw('mpl')
